In [1]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path

import importlib  
fa_etl = importlib.import_module("fa-etl")

In [2]:
county = "42101"

path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
taxhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet")
annual = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
sales = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")
merged = pl.read_parquet(path_to_dir+f"dev/{county}/unified/merged.parquet")

In [3]:
valhist.filter(
    pl.col('Year') > 2017
)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear,Year,Value,AssessmentUsed,RecentValueByYear
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,u32
116348448,0,2021,178800,2021,0,null,null,2021,178800,"""Market""",1
116348329,0,2021,45800,2021,0,null,null,2021,45800,"""Market""",1
116348332,0,2021,115800,2021,0,null,null,2021,115800,"""Market""",1
116347966,0,2021,202800,2021,0,null,null,2021,202800,"""Market""",1
116347972,0,2021,187000,2021,0,null,null,2021,187000,"""Market""",1
116348106,0,2019,249400,2020,0,null,null,2020,249400,"""Market""",1
116348056,0,2019,167600,2020,0,null,null,2020,167600,"""Market""",1
116348058,0,2019,285300,2020,0,null,null,2020,285300,"""Market""",1
116375262,787200,2020,787200,2020,0,null,null,2020,787200,"""Market""",1


In [4]:
merged.shape

(4531915, 28)

In [5]:
merged.head(10)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear,Year,Value,AssessmentUsed,RecentValueByYear,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,SaleAmt,TaxAmt,TaxAmtAdjusted
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,u32,str,date,f64,f64,str,str,str,str,str,str,str,str,str,i64,i64,f64
116348448,0,2021,178800,2021,0,null,null,2021,178800,"""Market""",1,"""R""",2024-01-19,39.936438,-75.163489,"""1204 PETERS ST…","""PHILADELPHIA""","""PA""","""19147""","""42101""","""002400""","""5013""","""B""","""42101_2400""",null,250284,2502.84
116348332,0,2021,115800,2021,0,null,null,2021,115800,"""Market""",1,"""R""",2024-01-19,39.93304,-75.160179,"""922 WHARTON ST…","""PHILADELPHIA""","""PA""","""19147""","""42101""","""002900""","""3000""","""B""","""42101_2900""",null,162097,1620.97
116347966,0,2021,202800,2021,0,null,null,2021,202800,"""Market""",1,"""R""",2024-01-19,39.926453,-75.161272,"""914 MCCLELLAN …","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1013""","""B""","""42101_2802""",null,283879,2838.79
116347972,0,2021,187000,2021,0,null,null,2021,187000,"""Market""",1,"""R""",2024-01-19,39.926482,-75.16156,"""926 MCCLELLAN …","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1013""","""B""","""42101_2802""",null,261763,2617.63
116348106,11072,2012,34600,2007,0,null,null,2012,11072,"""Assd""",1,"""R""",2024-01-19,39.928347,-75.159963,"""1631 S 9TH ST""","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1004""","""B""","""42101_2802""",null,null,null
116348106,187200,2014,187200,2015,0,null,null,2015,187200,"""Market""",1,"""R""",2024-01-19,39.928347,-75.159963,"""1631 S 9TH ST""","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1004""","""B""","""42101_2802""",null,1829131,18291.31
116348105,191900,2014,191900,2015,0,null,null,2015,191900,"""Market""",1,"""R""",2024-01-19,39.928392,-75.15996,"""1629 S 9TH ST""","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1004""","""B""","""42101_2802""",null,null,null
116348055,15904,2012,49700,2011,0,null,null,2012,15904,"""Assd""",1,"""R""",2024-01-19,39.928855,-75.15863,"""1612 S 8TH ST""","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1000""","""B""","""42101_2802""",null,null,null
116348055,269000,2014,269000,2015,0,null,null,2015,269000,"""Market""",1,"""R""",2024-01-19,39.928855,-75.15863,"""1612 S 8TH ST""","""PHILADELPHIA""","""PA""","""19148""","""42101""","""002802""","""1000""","""B""","""42101_2802""",null,2628399,26283.99


In [6]:
sales.filter(
    pl.col("RecordingYear") > 2017
)

PropertyID,SaleAmt,RecordingYear
i64,i64,i64
116347959,305000,2022
116347964,350000,2018
116347965,40000,2018
116347965,485000,2023
116347957,225000,2021
116347955,250000,2022
116347955,235000,2018
116347970,412500,2022
116347972,330000,2018


In [7]:
merged.filter([
    pl.col("SaleAmt").is_not_null(),
    pl.col("Year") > 2017
]).select(
    ['PropertyID','Year','Value','SaleAmt', 'TaxAmt', 'TaxAmtAdjusted']
).with_columns([
    (pl.col('TaxAmt')/pl.col('Value')).alias("ApproxTaxRate")
])

PropertyID,Year,Value,SaleAmt,TaxAmt,TaxAmtAdjusted,ApproxTaxRate
i64,i64,i64,i64,i64,f64,f64
116375325,2022,570200,295000,798166,7981.66,1.3998
116348295,2021,282900,385000,345611,3456.11,1.221672
116347981,2021,418800,371000,523245,5232.45,1.249391
116402418,2020,496800,549000,695421,6954.21,1.399801
116375282,2021,1812100,1750000,2536578,25365.78,1.3998
116388338,2018,137500,290000,192473,1924.73,1.399804
116388339,2018,154100,239000,173715,1737.15,1.127287
116388304,2018,41100,89000,57532,575.32,1.399805
116388343,2018,49600,1000,69430,694.3,1.399798
